In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style(
    style="darkgrid", 
    rc={"axes.facecolor": ".9", "grid.color": ".8"}
)
sns.set_palette(palette="deep")
sns_c = sns.color_palette(palette="deep")

import arviz as az
import patsy
import pymc3 as pm
from pymc3 import glm

plt.rcParams["figure.figsize"] = [7, 6]
plt.rcParams["figure.dpi"] = 100

In [ ]:
# install Anaconda3
!wget -qO ac.sh https://repo.anaconda.com/archive/Anaconda3-2020.07-Linux-x86_64.sh 
!bash ./ac.sh -b

# a fake google.colab library
!ln -s /usr/local/lib/python3.7/dist-packages/google \
       /root/anaconda3/lib/python3.8/site-packages/google

# start jupyterlab, which now has Python3 = 3.8
!nohup /root/anaconda3/bin/jupyter-lab --ip=0.0.0.0&

# access through ngrok, click the link
!pip install pyngrok -q
from pyngrok import ngrok
print(ngrok.connect(8888))


Generate Sample Data


In [ ]:
# Number of data points.
n = 250
# Create features.
x1 = np.random.normal(loc=0.0, scale=2.0, size=n)
x2 = np.random.normal(loc=0.0, scale=2.0, size=n)
epsilon = np.random.normal(loc=0.0, scale=0.5, size=n)
# Define target variable.
intercept = -0.5
beta_x1 = 1
beta_x2 = -1
beta_interaction = 2
z = intercept + beta_x1 * x1 + beta_x2 * x2 + beta_interaction * x1 * x2
p = 1 / (1 + np.exp(-z))
y = np.random.binomial(n=1, p=p, size=n)

df = pd.DataFrame(dict(x1=x1, x2=x2, y=y))

df.head()

In [ ]:
sns.pairplot(
    data=df,
    kind="scatter",
    height=2,
    plot_kws={"color": sns_c[1]},
    diag_kws={"color": sns_c[2]} )

x1 and x2 are not correlated.
x1 and x2 do not seem to separate the 
y-classes independently.
The distribution of y is not highly unbalanced.

In [ ]:
fig, ax = plt.subplots()
sns_c_div = sns.diverging_palette(240, 10, n=2)
sns.scatterplot(
  x="x1",
  y="x2",
  data=df,
  hue="y",
  palette=[sns_c_div[0], sns_c_div[-1]]
)
ax.legend(title="y", loc="center left", bbox_to_anchor=(1, 0.5))
ax.set(
  title="Sample Data",
  xlim=(-9, 9),
  ylim=(-9, 9),
  xlabel="x1",
  ylabel="x2"
);

Prepare Data for Modeling

In [ ]:
# Define model formula.
formula = "y ~ x1 * x2"
# Create features.
y, x = patsy.dmatrices(formula_like=formula, data=df)
y = np.asarray(y).flatten()
labels = x.design_info.column_names
x = np.asarray(x)
print(f"labels = {labels}")
labels = ['Intercept', 'x1', 'x2', 'x1:x2']

Now we do a train-test split

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
  x, y, train_size=0.7
)

Define the Model
We now specify the model in PyMC3.

In [ ]:
with pm.Model() as model:
    # Set data container.
    data = pm.Data("data", x_train)
    # Define GLM family.
    family = pm.glm.families.Binomial()
    # Set priors.
    priors = {
        "Intercept": pm.Normal.dist(mu=0, sd=10),
        "x1": pm.Normal.dist(mu=0, sd=10),
        "x2": pm.Normal.dist(mu=0, sd=10),
        "x1:x2": pm.Normal.dist(mu=0, sd=10),
    }
    # Specify model.
    glm.GLM(
      y=y_train,
      x=data,
      family=family,
      intercept=False,
      labels=labels,
      priors=priors
    )

Prior Checks
Before fitting the model we run some prior predictive checks on the training data.

In [ ]:
# Sample from prior distribution.
with model:
    prior_checks = pm.sample_prior_predictive(samples=100)
    prior_checks.keys()
prior_checks["y"]
train_prior_df = pd.DataFrame(
    data={
        "x1_train": x_train[:, 1],
        "x2_train": x_train[:, 2],
        "y_train": y_train,
        "y_train_prior_mean": prior_checks["y"].mean(axis=0),
    },
)

train_prior_df.sort_values("y_train", inplace=True)

In [ ]:
# Plot means distribution.
fig, ax = plt.subplots()
sns.kdeplot(
    x="y_train_prior_mean",
    data=train_prior_df,
    hue="y_train",
    palette=[sns_c[0], sns_c[3]],
    fill=True,
    alpha=0.1,
    ax=ax,
)
ax.axvline(x=0.5, color="gray", linestyle="--")
ax.set(title="Prior Means Distribution (train)", xlim=(0, 1));


We clearly see that the model can not distinguish between the two classes yet. This makes sense as we have non-informative priors for this synthetic data set. We can also confirm this if we plot each point separately:

In [ ]:
fig, ax = plt.subplots()
cmap = sns.diverging_palette(240, 10, n=50, as_cmap=True)
sns.scatterplot(
    x="x1_train",
    y="x2_train",
    data=train_prior_df,
    hue="y_train_prior_mean",
    hue_norm=(0, 1),
    palette=cmap,
    edgecolor="black",
    style="y_train",
    ax=ax,
)
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
ax.set(
  title="Prior Means (train)",
  xlim=(-9, 9),
  ylim=(-9, 9),
  xlabel="x1",
  ylabel="x2")

Fit Model


In [ ]:
 with model:
    # Configure sampler.
    trace = pm.sample(5000, chains=5, tune=1000, target_accept=0.87)
# Plot chains.
az.plot_trace(data=trace)
az.summary(trace)

In [ ]:
# Update data reference.
pm.set_data({"data": x_test}, model=model)
# Generate posterior samples.
ppc_test = pm.sample_posterior_predictive(trace, model=model, samples=1000)
# Compute the point prediction by taking the mean
# and defining the category via a threshold.
p_test_pred = ppc_test["y"].mean(axis=0)
y_test_pred = (p_test_pred >= 0.5).astype("int")

Evaluate Model

In [ ]:
#First let us compute the accuracy on the test set.

from sklearn.metrics import accuracy_score

print(f"accuracy = {accuracy_score(y_true=y_test, y_pred=y_test_pred): 0.3f}")
accuracy =  0.787
#Next, we plot the roc curve and compute the auc.

from sklearn.metrics import RocCurveDisplay, auc, roc_curve

fpr, tpr, thresholds = roc_curve(
    y_true=y_test, y_score=p_test_pred, pos_label=1, drop_intermediate=False
)
roc_auc = auc(fpr, tpr)

fig, ax = plt.subplots()
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
roc_display = roc_display.plot(ax=ax, marker="o", color=sns_c[4], markersize=4)
ax.set(title="ROC");